# J3: JSSP Dynamic (Disruptions) - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**J3: JSSP Dynamic (Disruptions)** - Job-shop scheduling with machine breakdowns and dynamic replanning.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for J3: JSSP Dynamic (Disruptions)


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Job & Machine Setup Agent ---- #
    JM_Agent = Agent(
        name="Job & Machine Setup Agent",
        backstory="You define jobs, machines, processing times, and initial schedule for dynamic JSSP.",
        task_description="Set up JSSP instance with jobs, machines, processing times, and create initial baseline schedule.",
        task_expected_output="Structured JSSP data with initial schedule: jobs, machines, processing times, and baseline schedule."
    )
    
    # ---- Disruption Detection Agent ---- #
    DD_Agent = Agent(
        name="Disruption Detection Agent",
        backstory="You monitor the shop floor and detect disruptions such as machine breakdowns and material shortages.",
        task_description="Detect disruptions: identify machine breakdowns (machine_id, duration), material shortages (resource, shortage percentage), and other unexpected events.",
        task_expected_output="Disruption report: type of disruption, affected machines/resources, duration/impact, and affected operations."
    )
    
    # ---- Impact Analysis Agent ---- #
    IA_Agent = Agent(
        name="Impact Analysis Agent",
        backstory="You analyze the impact of disruptions on the current schedule and identify affected operations.",
        task_description="Analyze disruption impact: identify operations affected by machine breakdowns or resource shortages, estimate delay impacts, and determine rescheduling needs.",
        task_expected_output="Impact analysis: list of affected operations, estimated delays, makespan impact, and operations requiring rescheduling."
    )
    
    # ---- Replanning Agent ---- #
    RP_Agent = Agent(
        name="Replanning Agent",
        backstory="You replan the schedule to accommodate disruptions while minimizing makespan increase and maintaining feasibility.",
        task_description="Replan schedule: reschedule affected operations, reassign to alternative machines if available, adjust operation sequences, and minimize makespan increase.",
        task_expected_output="Replanned schedule with updated operation times, machine reassignments, and new makespan."
    )
    
    # ---- Recovery Optimization Agent ---- #
    RO_Agent = Agent(
        name="Recovery Optimization Agent",
        backstory="You optimize the recovery plan to minimize disruption impact and restore schedule quality.",
        task_description="Optimize recovery: fine-tune replanned schedule to minimize makespan increase, balance machine workload, and ensure all constraints are satisfied.",
        task_expected_output="Optimized recovery schedule with minimal makespan increase, balanced workload, and full constraint satisfaction."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './j3_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    JM_Agent >> DD_Agent >> IA_Agent >> RP_Agent >> RO_Agent >> Writer_agent


In [ ]:
crew.run()
